This colab allows a user to extract all video files in a specified Google Drive.
The videos are returned as video player links for easy viewing.

In [32]:
# This should be the Public Google Drive Folder ID that contains all workout videos.
GOOGLE_DRIVE_WORKOUT_ID = '1U7FYKeoD669CtNtN64PBI4-tPkTlW05p' #@param {type: 'string'}

In [33]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [34]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
workout_folder_ids = set()
for file in drive.ListFile({'q': f"'{GOOGLE_DRIVE_WORKOUT_ID}' in parents",}).GetList():
    print('title {}, id {}'.format(file['title'], file['id']))
    workout_folder_ids.add(file['id'])


title russian twist, id 1qOHaHj5S1mbbSlFkPgGAXo1LBS4zcxSx
title romanian deadlift, id 1zVlLga5ACkicT6LglbzkmlgdSvs0mDE5
title hip thrust, id 1RxdAN-9ydzPpT02g53GSVX8xHwMdm1hn
title shoulder press, id 1lLWuVjAYiZyzmXy3gq75NwE4URl91wgs
title pull Up, id 1bfQluf50neeBei_JvZL64Zq69d2o0r7g
title plank, id 1LkbIvJvlQZliRPpPIdKnT4_RYl7Z1bJA
title bench press, id 13SbUgXnglw6syYywfnI-kwuXE3vqx82O
title decline bench press, id 1x_Q6br5hrslDXO4TCbooo313-UixcUxp
title leg raises, id 1qtduCup5jyayWO3zWwGGJv0YSbJkAyf9
title lateral raise, id 1Z6MiSUsCr7ZW4QmIb8LLMzTMABk1eBLZ
title leg extension, id 10SwcRU9HkzAZZuy-usKUmPQYdu_zDFr4
title t bar row, id 1Bww-S0NTE2FYrhi0lrf-dUozFxN7Rzqu
title incline bench press, id 1yh2V89BPzg735DcV7r-LIhxPwwIlQigJ
title deadlift, id 11tF2CPVp_z_20ifsiiE9_oCjzcwLuk99
title chest fly machine, id 1bYtOce16YFf1o4dGgeMClNsFJeDtbHtZ
title tricep Pushdown, id 1ScY9hAOaFYGBjPSGzyQwFY34_f4v5VLW
title barbell biceps curl, id 1msW4xUpSvubKWZnajnJuxQv0ix8hAISE
title squat, id 

In [35]:
def get_video_link(id: str) -> str:
  '''
  Parses the Google Drive Link given a folder ID.
  '''
  return f"https://drive.google.com/file/d/{id}/view?usp=drive_link"


for id in workout_folder_ids:
  for file in drive.ListFile({'q': f"'{id}' in parents",}).GetList():
    print('Video Title: {}, Video Link: {}'.format(
        file['title'], get_video_link(file['id'])))

Video Title: t bar row_8.mp4, Video Link: https://drive.google.com/file/d/1NJ7gZgW6NRHg1QISwG9kb9lM6BNQr2zP/view?usp=drive_link
Video Title: t bar row_9.mp4, Video Link: https://drive.google.com/file/d/1rsYjCpBIuKYbCrAtq4qp1eiD_UmtqnON/view?usp=drive_link
Video Title: t bar row_7.MOV, Video Link: https://drive.google.com/file/d/1_YZozzv7lnpyp7Vc2D4Vq-BoMOBeTzwj/view?usp=drive_link
Video Title: t bar row_6.MOV, Video Link: https://drive.google.com/file/d/1lq5owjdaenFZt4fu67uqbGUEmCpQJ85O/view?usp=drive_link
Video Title: t bar row_5.MOV, Video Link: https://drive.google.com/file/d/1i4nhiw2_oUoTQdLs9Og28-dCFgQtiejg/view?usp=drive_link
Video Title: t bar row_4.MOV, Video Link: https://drive.google.com/file/d/1ZUlqh8M8i3xtctFFsLYBAT1Nac0Xsv6y/view?usp=drive_link
Video Title: t bar row_3.MOV, Video Link: https://drive.google.com/file/d/1vQsl7Ugt5EE6_5bNkpqarcRj951NrYKD/view?usp=drive_link
Video Title: t bar row_20.mp4, Video Link: https://drive.google.com/file/d/132B_V9ruSzvuyMH0JJ1-aLfle_mk